# Importing Dependences

In [1]:
!pip install gdown

In [2]:
import pandas as pd
import numpy as np 
import os
import gdown
import zipfile
import shutil
import re

In [3]:
gdrive_url = 'https://drive.google.com/uc?id=1jmSta-CD03w341lOzNjT_1qAeNDt2zL2'
output_path = 'archive.zip'
output_folder_path = "archive"
extract_to_folder = "c:\\temp_extract"

# Loading Dataset

In [4]:
def extract_zip(zip_file_path, extract_to):
    # Extract the zip file if it exists and is valid
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
            print(f"Extracted all files to {extract_to}")
    except FileNotFoundError:
        print(f"Error: File {zip_file_path} does not exist.")
    except zipfile.BadZipFile:
        print("Error: The file is not a valid zip file.")
    except Exception as e:
        print(f"An error occurred: {e}")

def download_and_extract(gdrive_url, output_path, extract_to_folder, output_folder_path):
    # Download the file
    try:
        gdown.download(gdrive_url, output_path, quiet=False)
        print(f"File downloaded and saved as {output_path}")
        
        # Extract the downloaded zip file
        extract_zip(output_path, extract_to_folder)

        # Move the extracted folder and clean up
        shutil.move(extract_to_folder, output_folder_path)
        os.remove(output_path)
        print(f"Moved extracted files to {output_folder_path} and removed {output_path}")
    except Exception as e:
        print(f"An error occurred during the download or extraction process: {e}")

download_and_extract(gdrive_url, output_path, extract_to_folder, output_folder_path)

Downloading...
From (original): https://drive.google.com/uc?id=1jmSta-CD03w341lOzNjT_1qAeNDt2zL2
From (redirected): https://drive.google.com/uc?id=1jmSta-CD03w341lOzNjT_1qAeNDt2zL2&confirm=t&uuid=5f236f5c-9ca1-4de7-aab1-7941e0363a49
To: c:\Users\syahr\OneDrive\Desktop\Portfolio\36118_NLP_Spring\archive.zip
100%|██████████| 689M/689M [01:58<00:00, 5.80MB/s] 


File downloaded and saved as archive.zip
Extracted all files to c:\temp_extract
An error occurred during the download or extraction process: Destination path 'archive\temp_extract' already exists


# Data Exploration

In [4]:
# Define the correct path for the screenplay data
texts_path = os.path.join(output_folder_path, "screenplay_data", "data", "raw_texts", "raw_texts")

# Initialize dictionary to store file names and contents
screenplays = {}

# List and iterate over all files in the folder
for file_name in os.listdir(texts_path):
    file_path = os.path.join(texts_path, file_name)
    
    # Ensure the path is an actual file before reading
    if os.path.isfile(file_path):
        
        try:
            # Read and store file content
            with open(file_path, 'r', encoding='latin-1') as f:
                screenplays[file_name] = f.read()
        except Exception as e:
            print(f"Error reading {file_name}: {e}")

# Print a sample of the first ten files to check
for i, (file_name, content) in enumerate(screenplays.items()):
    if i == 10:
        break
    print(f"Example of {file_name}:\n")
    print(content[:100])  # Print the first 100 characters as a sample
    print("-" * 50)

Example of 10 Cloverfield Lane_1179933.txt:

The Cellar

by
Josh Campbell & Matt Stuecken
DARKNESS

And then --

A GUNNED ENGINE --

BLURRED HEA
--------------------------------------------------
Example of 10 Things I Hate About You_0147800.txt:


                               TEN THINGS I HATE ABOUT YOU
          
                written by Ka
--------------------------------------------------
Example of 101 Days of 101 Dalmatians_0249328.txt:

107
#2
40] _DALMATIANS MARCH 17, 1995

EXT. SKY. FULL MOON

A huge, yellow moon. CAMERA TILTS DOWN 
--------------------------------------------------
Example of 12 Angry Men_0118528.txt:

PLEASE COPY AND RETURN |

âââ_ââââ_

 

TWELVE ANGRY MEN

by Reginald Rose

THE WRITIN
--------------------------------------------------
Example of 12 Monkeys_0114746.txt:


				TWELVE MONKEYS
	    
		          An original screenplay by

				David Peoples
  				     &amp;

--------------------------------------------------
Example of 12 Yea

In [5]:
# Set display option for better visibility
pd.set_option('display.max_columns', 25)

# Define the path to the metadata CSV file
csv_path = os.path.join(output_folder_path, 'movie_metadata', 'movie_meta_data.csv')

# Ensure the file exists before trying to read
if os.path.exists(csv_path):
    # Read the CSV file
    meta_df = pd.read_csv(csv_path)
    
    # Print the column names
    print(meta_df.columns)
    
    # Display the first few rows of the dataframe
    print(meta_df.head())
else:
    print(f"File {csv_path} does not exist.")

Index(['imdbid', 'title', 'akas', 'year', 'metascore', 'imdb user rating',
       'number of imdb user votes', 'awards', 'opening weekend', 'producers',
       'budget', 'script department', 'production companies', 'writers',
       'directors', 'casting directors', 'cast', 'countries', 'age restrict',
       'plot', 'plot outline', 'keywords', 'genres', 'taglines', 'synopsis'],
      dtype='object')
   imdbid                   title  \
0  120770  A Night at the Roxbury   
1  132512          At First Sight   
2  118661            The Avengers   
3  215545              Bamboozled   
4  118715        The Big Lebowski   

                                                akas  year  metascore  \
0  Une nuit au Roxbury (France), Movida en el Rox...  1998         26   
1  Sight Unseen (United States), Premier regard (...  1999         40   
2  Chapeau melon et bottes de cuir (France), Mit ...  1998         12   
3  The Very Black Show (France), It's Showtime (G...  2000         54   
4  El gr

The columns of interest include:

- Title
- Age restriction
- Year: This could be useful for analyzing shifts in cultural norms over time. For instance, certain curse words may have led to an MA rating in the 1960s but not in the 2020s.
- Budget and Opening weekend: These may be important for studying the effect of movie classification on its financial performance.
- IMDb ID: This might be relevant for linking additional data from the IMDb database.

In [6]:
# Extract movie titles and IDs in one step using list comprehension
movie_titles, ids = zip(*[(f.split("_", 1)[0], f.split("_", 1)[1].split(".", 1)[0]) for f in screenplays.keys()])

# Print the first 10 titles and IDs
for i, (title, id) in enumerate(zip(movie_titles, ids)):
    if i == 10:
        break
    print(f"Title: {title}  ID: {id}")

Title: 10 Cloverfield Lane  ID: 1179933
Title: 10 Things I Hate About You  ID: 0147800
Title: 101 Days of 101 Dalmatians  ID: 0249328
Title: 12 Angry Men  ID: 0118528
Title: 12 Monkeys  ID: 0114746
Title: 12 Years a Slave  ID: 2024544
Title: 127 Hours  ID: 1542344
Title: 13 13 13  ID: 2991516
Title: 1408  ID: 0450385
Title: 1492 Conquest of Paradise  ID: 0103594


# Dataset Preprocessing

In [7]:
# Create a DataFrame directly from screenplays, extracting ids from the keys
screenplays_df = pd.DataFrame({
    'imdbid': [os.path.splitext(f.split("_", 1)[1])[0].replace(".txt", "") for f in screenplays.keys()],
    'screenplay': screenplays.values()
})

screenplays_df.head()

,imdbid,screenplay
0,1179933,The Cellar\n\nby\nJosh Campbell & Matt Stuecke...
1,0147800,\n TEN THINGS I ...
2,0249328,"107\n #2\n40] _DALMATIANS MARCH 17, 1995\n\nEX..."
3,0118528,PLEASE COPY AND RETURN |\n\nâââ_âââ...
4,0114746,\n\t\t\t\tTWELVE MONKEYS\n\t \n\t\t ...


In [8]:
print(meta_df.info())
print(screenplays_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2858 entries, 0 to 2857
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   imdbid                     2858 non-null   int64 
 1   title                      2858 non-null   object
 2   akas                       2652 non-null   object
 3   year                       2858 non-null   int64 
 4   metascore                  2858 non-null   int64 
 5   imdb user rating           2858 non-null   int64 
 6   number of imdb user votes  2858 non-null   int64 
 7   awards                     2243 non-null   object
 8   opening weekend            1739 non-null   object
 9   producers                  2640 non-null   object
 10  budget                     1624 non-null   object
 11  script department          2220 non-null   object
 12  production companies       2682 non-null   object
 13  writers                    2696 non-null   object
 14  director

In [9]:
# Ensure 'imdbid' is of the correct type in both DataFrames before merging, if needed
if screenplays_df['imdbid'].dtype != 'int':
    screenplays_df['imdbid'] = screenplays_df['imdbid'].astype(int)

# Merge with metadata on 'imdbid'
df = meta_df.merge(screenplays_df, on='imdbid')

df.head()

,imdbid,title,akas,year,metascore,imdb user rating,number of imdb user votes,awards,opening weekend,producers,budget,script department,...,directors,casting directors,cast,countries,age restrict,plot,plot outline,keywords,genres,taglines,synopsis,screenplay
0,120770,A Night at the Roxbury,"Une nuit au Roxbury (France), Movida en el Rox...",1998,26,6,56537,NaN,United States:,"Marie Cantin, Erin Fraser, Amy Heckerling, Ste...","$17,000,000 (estimated)",NaN,...,John Fortenberry,Jeff Greenberg,"Will Ferrell, Chris Kattan, Raquel Gardner, Vi...",United States,"Argentina:13, Australia:M, Brazil:14, Canada:P...",Two dim-witted brothers dream of owning their ...,"The Roxbury Guys, Steve and Doug Butabi, want ...","woman-on-top, nightclub, car-accident, 1990s, ...","Comedy, Music, Romance",Score!,NaN,\n\n\t\t\t A NIGHT AT THE ROXBURY \n\n\n\t\...
1,132512,At First Sight,"Sight Unseen (United States), Premier regard (...",1999,40,6,12922,NaN,United States:,"Rob Cowan, Roger Paradiso, Irwin Winkler","$60,000,000 (estimated)",NaN,...,Irwin Winkler,"Kerry Barden, Billy Hopkins, Suzanne Smith","Val Kilmer, Mira Sorvino, Kelly McGillis, Stev...",United States,"Argentina:13, Australia:M, Canada:PG::(Alberta...",A blind man has an operation to regain his sig...,First Sight is true to the title from start to...,"visual-agnosia, brother-sister-relationship, r...","Drama, Romance","Only Love Can Bring You To Your Senses., Scien...",NaN,AT FIRST SIGHT\n\nEXT. VALLEY - DUSK \nGold li...
2,118661,The Avengers,"Chapeau melon et bottes de cuir (France), Mit ...",1998,12,3,40784,"FMCJ Award 1998, Golden Reel Award 1999, Razzi...","United States: $10,305,957, 16 Aug 1998","Susan Ekins, Jerry Weintraub","$60,000,000 (estimated)","Sharon Mansfield, Anna Worley",...,Jeremiah S. Chechik,Susie Figgis,"Ralph Fiennes, Uma Thurman, Sean Connery, Patr...",United States,"Argentina:13, Australia:PG, Brazil:10, Canada:...",Two British Agents team up to stop Sir August ...,"British Ministry Agent John Steed, under direc...","good-versus-evil, heroine, evil-man, villain, ...","Action, Adventure, Sci-Fi, Thriller","Mrs. Peel, we're needed., Extraordinary crimes...",NaN,\n\n\t\t\t\t\tTHE AVENGERS\n\n\t\t\t\tScreenpl...
3,215545,Bamboozled,"The Very Black Show (France), It's Showtime (G...",2000,54,6,10373,"Golden Berlin Bear 2001, Black Reel 2001, Imag...",United States:,"Jon Kilik, Spike Lee, Kisha Imani Cameron","$10,000,000 (estimated)","Shari L. Carpenter, Carolyn De Sousa",...,Spike Lee,Aisha Coley,"Damon Wayans, Savion Glover, Jada Pinkett Smit...",United States,"Australia:MA, Finland:K-15, France:Tous public...",A frustrated African-American TV writer propos...,"Dark, biting satire of the television industry...","television-industry, african-american, referen...","Comedy, Drama, Music",Starring the great negroe actors,"In a New York City residence, Pierre Delacroix...",\t\t\t\tBamboozled\n\n\t\t\t\tby\n\n\t\t\t\tSp...
4,118715,The Big Lebowski,"El gran Lebowski (Spain), O Grande Lebowski (P...",1998,71,8,724388,"Honorable Mention 1998, ACCA 1998, Golden Berl...","United States: $5,533,844, 08 Mar 1998","Tim Bevan, John Cameron, Ethan Coen, Eric Fell...","$15,000,000 (estimated)",T. Kukovinski,...,"Joel Coen, Ethan Coen",John S. Lyons,"Jeff Bridges, John Goodman, Julianne Moore, St...","United States, United Kingdom","Argentina:16, Argentina:18::(cable rating), Au...","Jeff ""The Dude"" Lebowski, mistaken for a milli...","When ""the dude"" Lebowski is mistaken for a mil...","rug, nihilism, pornographer, bowling-alley, de...","Comedy, Crime, Sport",Hay quienes tratan de ganarse la vida sin move...,A tumbleweed rolls up a hillside just outside ...,\n\n\t\t\tTHE BIG LEBOWSKI\n\nWe are floating ...


In [10]:
# Create a lean version of the dataframe with only relevant columns for predicting age restrict classification
df_lean = df[['imdbid', 'title', 'year', 'opening weekend', 'budget', 'age restrict', 'genres', 'screenplay']]
df_lean.head()

,imdbid,title,year,opening weekend,budget,age restrict,genres,screenplay
0,120770,A Night at the Roxbury,1998,United States:,"$17,000,000 (estimated)","Argentina:13, Australia:M, Brazil:14, Canada:P...","Comedy, Music, Romance",\n\n\t\t\t A NIGHT AT THE ROXBURY \n\n\n\t\...
1,132512,At First Sight,1999,United States:,"$60,000,000 (estimated)","Argentina:13, Australia:M, Canada:PG::(Alberta...","Drama, Romance",AT FIRST SIGHT\n\nEXT. VALLEY - DUSK \nGold li...
2,118661,The Avengers,1998,"United States: $10,305,957, 16 Aug 1998","$60,000,000 (estimated)","Argentina:13, Australia:PG, Brazil:10, Canada:...","Action, Adventure, Sci-Fi, Thriller",\n\n\t\t\t\t\tTHE AVENGERS\n\n\t\t\t\tScreenpl...
3,215545,Bamboozled,2000,United States:,"$10,000,000 (estimated)","Australia:MA, Finland:K-15, France:Tous public...","Comedy, Drama, Music",\t\t\t\tBamboozled\n\n\t\t\t\tby\n\n\t\t\t\tSp...
4,118715,The Big Lebowski,1998,"United States: $5,533,844, 08 Mar 1998","$15,000,000 (estimated)","Argentina:16, Argentina:18::(cable rating), Au...","Comedy, Crime, Sport",\n\n\t\t\tTHE BIG LEBOWSKI\n\nWe are floating ...


In [11]:
df_lean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2853 entries, 0 to 2852
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   imdbid           2853 non-null   int64 
 1   title            2853 non-null   object
 2   year             2853 non-null   int64 
 3   opening weekend  1736 non-null   object
 4   budget           1623 non-null   object
 5   age restrict     2524 non-null   object
 6   genres           2841 non-null   object
 7   screenplay       2853 non-null   object
dtypes: int64(2), object(6)
memory usage: 178.4+ KB


In [12]:
# Assess missing values in 'age restrict' and print the shape in a single step
missing_values = df_lean['age restrict'].isnull().sum()
df_lean.shape, missing_values

((2853, 8), 329)

In [13]:
# Drop rows with missing values in 'age restrict' and check the shape in a single step
df_clean = df_lean.dropna(subset=['age restrict'])
df_clean.shape, df_clean['age restrict'].isnull().sum()

((2524, 8), 0)

In [26]:
import re
import pandas as pd

# Define the regex pattern for Australia
pattern = re.compile(r"Australia:([A-Za-z0-9-]+)")

# Function to find the Australia classification in a string
def find_aus_classification(string):
    if isinstance(string, str):  # Ensure the input is a string
        match = pattern.search(string)
        return match.group(1) if match else pd.NA  # Return the matched value or NA
    return pd.NA  # Return NA if the input is not a string

# Assuming df_clean is your DataFrame and 'age restrict' is the column of interest
aus_classifications = df_clean['age restrict'].apply(find_aus_classification)

# Output the classifications for Australia
aus_classifications



0          M
1          M
2         PG
3         MA
4         MA
        ... 
2848    MA15
2849      PG
2850    MA15
2851      PG
2852    MA15
Name: age restrict, Length: 2524, dtype: object

In [15]:
# Create a dataset with Australian classifications
df_clean['age restrict aus'] = aus_classifications
df_clean.head()

C:\Users\syahr\AppData\Local\Temp\ipykernel_11720\58119831.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['age restrict aus'] = aus_classifications


,imdbid,title,year,opening weekend,budget,age restrict,genres,screenplay,age restrict aus
0,120770,A Night at the Roxbury,1998,United States:,"$17,000,000 (estimated)","Argentina:13, Australia:M, Brazil:14, Canada:P...","Comedy, Music, Romance",\n\n\t\t\t A NIGHT AT THE ROXBURY \n\n\n\t\...,M
1,132512,At First Sight,1999,United States:,"$60,000,000 (estimated)","Argentina:13, Australia:M, Canada:PG::(Alberta...","Drama, Romance",AT FIRST SIGHT\n\nEXT. VALLEY - DUSK \nGold li...,M
2,118661,The Avengers,1998,"United States: $10,305,957, 16 Aug 1998","$60,000,000 (estimated)","Argentina:13, Australia:PG, Brazil:10, Canada:...","Action, Adventure, Sci-Fi, Thriller",\n\n\t\t\t\t\tTHE AVENGERS\n\n\t\t\t\tScreenpl...,PG
3,215545,Bamboozled,2000,United States:,"$10,000,000 (estimated)","Australia:MA, Finland:K-15, France:Tous public...","Comedy, Drama, Music",\t\t\t\tBamboozled\n\n\t\t\t\tby\n\n\t\t\t\tSp...,<NA>
4,118715,The Big Lebowski,1998,"United States: $5,533,844, 08 Mar 1998","$15,000,000 (estimated)","Argentina:16, Argentina:18::(cable rating), Au...","Comedy, Crime, Sport",\n\n\t\t\tTHE BIG LEBOWSKI\n\nWe are floating ...,MA


In [16]:
# Drop rows with missing values in 'age restrict aus' and output the shape
df_aus = df_clean.dropna(subset=['age restrict aus'])
df_aus.shape

(1891, 9)

M (Mature) - Assigned a value of 0, this rating indicates content that is recommended for audiences aged 15 and above. It may contain moderate themes, violence, or coarse language. Viewer discretion is advised for younger audiences.

PG (Parental Guidance) - With a value of 1, this rating suggests that some content may not be suitable for children without the guidance of a parent or guardian. Themes, language, and scenes are generally mild, but certain elements may require explanation or oversight.

MA (Mature Audience) - Rated 2, this classification is for mature viewers aged 15 years and above, and may include strong themes, violence, or explicit language. Parental guidance is strongly recommended for younger viewers.

G (General) - Rated 3, this classification means the content is suitable for all audiences. There are no themes, language, or visuals that would be inappropriate for children, making it family-friendly.

R (Restricted) - Assigned a value of 4, this rating is for adults only (18+). It includes content that may feature explicit language, violence, sexual content, or other strong themes that are not suitable for younger audiences.

MA15+ (Mature Audience 15+) - With the highest rating value of 5, this classification is specifically for people aged 15 and above. Content may include strong themes, violence, and other mature subject matter. Viewer discretion is required, and it is unsuitable for younger audiences.

# Modelling

In [36]:
df_aus["cleaned_screenplay"].iloc[-2]

"By Pat Proft  41197  51597 \x0c  The following dramatization is true, based on real events, from other actual movies. Building underneath the super  A  off in the distance.      hears the sirens. Beautiful, cool, in control, the girl nextdoor with the body of a Victoria's Secret model. And an air of mystery about her. She grabs her binoculars.    The penitentiary with a person jumping over the wall. Cass looks through her binocs.       The side of the truck reads A Prison Truck. In the b.g. off in the distance is the penitentiary where a break has just been discovered. In the flatbed are barrels, over stuffed, and marked Recycled Paper, Recycled Cans, Recycled Plastic, and Recycled Cycles, which is full of old bicycles. One barrel, which is marked For Escaping. has a pair of hands holding onto the rim. The person inside starts to rock the barrel until it falls off the truck.    Rolls down a hill. Passes a sign The Steepest Hill In The Whole World.   The world is spinning as we roll do

In [37]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import RandomOverSampler

# Precompile regular expressions for efficiency
newline_pattern = re.compile(r'\n+')
tab_pattern = re.compile(r'\t+')
space_pattern = re.compile(r' +')
screenplay_formatting_pattern = re.compile(r'(FADE IN:|CUT TO:|EXT\.|INT\.|SUPERIMPOSE:)', re.IGNORECASE)
meta_data_pattern = re.compile(r'(written by|&|June \d{1,2}, \d{4})', re.IGNORECASE)
all_caps_pattern = re.compile(r'[A-Z]{2,}(?:\s+[A-Z]{2,})*')

# Function to clean the text
def clean_screenplay_text(text):
    # Remove form feed characters (\x0c)
    text = text.replace('\x0c', '')
    # Replace multiple newlines with a single newline
    text = newline_pattern.sub(' ', text)
    # Replace tabs with a single space
    text = tab_pattern.sub(' ', text)
    # Replace multiple spaces with a single space
    text = space_pattern.sub(' ', text)
    # Remove screenplay formatting (e.g., FADE IN:, CUT TO:, etc.)
    text = screenplay_formatting_pattern.sub('', text)
    # Remove metadata like author names or dates
    text = meta_data_pattern.sub('', text)
    # Remove all caps, typically used for scene descriptions
    text = all_caps_pattern.sub('', text)
    # Remove special characters that are not part of standard sentences
    text = re.sub(r'[^A-Za-z0-9\'\.\?\!,\s]', '', text)
    # Standardize single quotes
    text = re.sub(r'[‘’]', "'", text)
    # Standardize double quotes
    text = re.sub(r'[“”]', '"', text)
    # Remove leading/trailing spaces
    text = text.strip()
    return text

# Assuming df_aus is a DataFrame with "screenplay" and "age restrict aus" columns
print("Dropping rows with NaN in 'screenplay' or 'age restrict aus' columns...")
df_aus = df_aus.dropna(subset=['screenplay', 'age restrict aus'])  # Drop rows with NaNs

print(f"Cleaning {len(df_aus)} screenplays...")
df_aus['cleaned_screenplay'] = df_aus['screenplay'].apply(clean_screenplay_text)

# Preprocessing: Vectorize the screenplays
print("Vectorizing the cleaned screenplays...")
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = tfidf_vectorizer.fit_transform(df_aus['cleaned_screenplay'])
print(f"Vectorization complete. Feature matrix shape: {X.shape}")

# Map the ratings to numerical values
rating_mapping = {'G': 0, 'PG': 1, 'M': 2, 'MA': 3, 'MA15+': 4, 'R': 5}
print(f"Mapping age restriction labels to numerical values...")
df_aus['mapped_rating'] = df_aus['age restrict aus'].map(rating_mapping)

# Drop any remaining NaN values in 'mapped_rating'
print("Ensuring data consistency...")
df_aus = df_aus.dropna(subset=['mapped_rating'])

# Ensure X is aligned with df_aus by slicing X with the same length as df_aus
X = X[:len(df_aus)]
y = df_aus['mapped_rating']
print(f"Data cleaned. Total records after processing: {len(df_aus)}")

# Split the dataset before oversampling
print("Splitting data into training and testing sets (80/20 split)...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Training data: {X_train.shape}, Test data: {X_test.shape}")

# Oversample the training data
print("Applying oversampling to balance the classes in the training data...")
oversampler = RandomOverSampler(random_state=42)
X_train, y_train = oversampler.fit_resample(X_train, y_train)
print(f"After oversampling, training data shape: {X_train.shape}")

# Train the classifier
print("Training Logistic Regression model...")
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
print("Model training complete.")

# Predict on the test set
print("Making predictions on the test set...")
y_pred = model.predict(X_test)

# Evaluation
print("Evaluating model performance...")
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", classification_report(
    y_test, y_pred, target_names=list(rating_mapping.keys()))
)

# Function to predict the rating of new screenplays
def predict_movie_rating(screenplay):
    print(f"Predicting rating for a new screenplay: '{screenplay[:30]}...'")
    screenplay_cleaned = clean_screenplay_text(screenplay)  # Clean new screenplay
    screenplay_vector = tfidf_vectorizer.transform([screenplay_cleaned])
    rating_index = model.predict(screenplay_vector)[0]
    # Retrieve the rating label from the mapping
    return {v: k for k, v in rating_mapping.items()}[rating_index]

# Example usage: Provide a new screenplay text (instead of passing a DataFrame column)
new_screenplay_text, actual_rating = df_aus[["screenplay", "age restrict aus"]].iloc[0]
predicted_rating = predict_movie_rating(new_screenplay_text)
print(f"Predicted Rating: {predicted_rating}")
print(f"Actual Rating: {actual_rating}")

Dropping rows with NaN in 'screenplay' or 'age restrict aus' columns...
Cleaning 1891 screenplays...
Vectorizing the cleaned screenplays...
Vectorization complete. Feature matrix shape: (1891, 5000)
Mapping age restriction labels to numerical values...
Ensuring data consistency...
Data cleaned. Total records after processing: 1891
Splitting data into training and testing sets (80/20 split)...
Training data: (1512, 5000), Test data: (379, 5000)
Applying oversampling to balance the classes in the training data...
After oversampling, training data shape: (4206, 5000)
Training Logistic Regression model...
Model training complete.
Making predictions on the test set...
Evaluating model performance...
Accuracy: 0.3905

Classification Report:
               precision    recall  f1-score   support

           G       0.20      0.33      0.25        18
          PG       0.52      0.52      0.52        67
           M       0.54      0.43      0.48       176
          MA       0.15      0.17    

In [38]:
# Custom accuracy function that considers predictions correct if the difference is at most 1
def custom_accuracy(y_true, y_pred):
    correct = sum(abs(y_t - y_p) <= 1 for y_t, y_p in zip(y_true, y_pred))
    return correct / len(y_true)

# Custom precision, recall, and F1-score functions for relaxed criteria
def custom_f1_score(y_true, y_pred, label):
    # For each label, compute custom precision, recall, and F1 score

    # True positives: predictions within 1 of the actual value
    tp = sum(abs(y_t - y_p) <= 1 and y_t == label for y_t, y_p in zip(y_true, y_pred))
    
    # False positives: predicted as this label but the actual value is more than 1 away
    fp = sum(abs(y_t - y_p) > 1 and y_p == label for y_t, y_p in zip(y_true, y_pred))
    
    # False negatives: actual value is this label, but the predicted value is more than 1 away
    fn = sum(abs(y_t - y_p) > 1 and y_t == label for y_t, y_p in zip(y_true, y_pred))
    
    # Compute precision, recall, and F1 score
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# Evaluating model performance
print("Evaluating model performance...")

# Use custom accuracy instead of default accuracy_score
accuracy = custom_accuracy(y_test, y_pred)
print(f"Custom Accuracy: {accuracy:.4f}")

# Get unique labels
unique_labels = np.unique(y_test)

# Evaluating model performance
print("Evaluating model performance...")

# Calculate and print custom F1 score for each label
for label in unique_labels:
    precision, recall, f1 = custom_f1_score(y_test, y_pred, label)
    print(f"Label: {label}")
    print(f"  Custom Precision: {precision:.4f}")
    print(f"  Custom Recall: {recall:.4f}")
    print(f"  Custom F1-Score: {f1:.4f}\n")

# Classification report for comparison (using the original metric)
print("\nClassification Report:\n", classification_report(
    y_test, y_pred, target_names=list(rating_mapping.keys()))
)

Evaluating model performance...
Custom Accuracy: 0.6755
Evaluating model performance...
Label: 0
  Custom Precision: 0.4000
  Custom Recall: 0.5556
  Custom F1-Score: 0.4651

Label: 1
  Custom Precision: 0.8493
  Custom Recall: 0.9254
  Custom F1-Score: 0.8857

Label: 2
  Custom Precision: 0.7800
  Custom Recall: 0.6648
  Custom F1-Score: 0.7178

Label: 3
  Custom Precision: 0.8667
  Custom Recall: 0.7429
  Custom F1-Score: 0.8000

Label: 4
  Custom Precision: 0.4730
  Custom Recall: 0.5556
  Custom F1-Score: 0.5109

Label: 5
  Custom Precision: 0.2222
  Custom Recall: 0.3000
  Custom F1-Score: 0.2553


Classification Report:
               precision    recall  f1-score   support

           G       0.20      0.33      0.25        18
          PG       0.52      0.52      0.52        67
           M       0.54      0.43      0.48       176
          MA       0.15      0.17      0.16        35
       MA15+       0.31      0.35      0.33        63
           R       0.12      0.20      0

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import classification_report
import numpy as np

# Precompile regular expressions for efficiency
newline_pattern = re.compile(r'\n+')
tab_pattern = re.compile(r'\t+')
space_pattern = re.compile(r' +')
screenplay_formatting_pattern = re.compile(r'(FADE IN:|CUT TO:|EXT\.|INT\.|SUPERIMPOSE:)', re.IGNORECASE)
meta_data_pattern = re.compile(r'(written by|&|June \d{1,2}, \d{4})', re.IGNORECASE)
all_caps_pattern = re.compile(r'[A-Z]{2,}(?:\s+[A-Z]{2,})*')

# Function to clean the text
def clean_screenplay_text(text):
    # Remove form feed characters (\x0c)
    text = text.replace('\x0c', '')
    # Replace multiple newlines with a single newline
    text = newline_pattern.sub(' ', text)
    # Replace tabs with a single space
    text = tab_pattern.sub(' ', text)
    # Replace multiple spaces with a single space
    text = space_pattern.sub(' ', text)
    # Remove screenplay formatting (e.g., FADE IN:, CUT TO:, etc.)
    text = screenplay_formatting_pattern.sub('', text)
    # Remove metadata like author names or dates
    text = meta_data_pattern.sub('', text)
    # Remove all caps, typically used for scene descriptions
    text = all_caps_pattern.sub('', text)
    # Remove special characters that are not part of standard sentences
    text = re.sub(r'[^A-Za-z0-9\'\.\?\!,\s]', '', text)
    # Standardize single quotes
    text = re.sub(r'[‘’]', "'", text)
    # Standardize double quotes
    text = re.sub(r'[“”]', '"', text)
    # Remove leading/trailing spaces
    text = text.strip()
    return text

# Preprocess screenplays and ratings
print("Cleaning screenplay texts...")
df_aus['cleaned_screenplay'] = df_aus['screenplay'].apply(clean_screenplay_text)
print(f"Sample cleaned screenplay: {df_aus['cleaned_screenplay'].iloc[0]}")

# Map ratings to numerical values
print("Mapping age restrictions to numerical values...")
rating_mapping = {'G': 0, 'PG': 1, 'M': 2, 'MA': 3, 'MA15+': 4, 'R': 5}
df_aus['mapped_rating'] = df_aus['age restrict aus'].map(rating_mapping)
print(f"Sample mapped rating: {df_aus['mapped_rating'].iloc[0]}")

# Drop rows with NaNs
print("Dropping rows with missing values...")
df_aus.dropna(subset=['cleaned_screenplay', 'mapped_rating'], inplace=True)
print(f"Data after dropping NaNs: {len(df_aus)} rows")

# Split data into features (X) and labels (y)
X = df_aus['cleaned_screenplay']
y = df_aus['mapped_rating']

# Split the data into training and test sets
print("Splitting data into training and test sets...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Training set size: {len(X_train)}, Test set size: {len(X_test)}")

# Apply RandomOverSampler to the training set
print("Applying RandomOverSampler to balance the training set...")
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train.values.reshape(-1, 1), y_train)

# Flatten the resampled data back to a Series
X_train_resampled = pd.Series(X_train_resampled.flatten())

# Tokenizer and model initialization
print("Loading BERT tokenizer and model...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(rating_mapping))

# Tokenize data
print("Tokenizing data...")
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

# Convert data to Hugging Face Dataset format
print("Converting data to Dataset format...")
train_dataset = Dataset.from_pandas(pd.DataFrame({'text': X_train_resampled, 'label': y_train_resampled}))
test_dataset = Dataset.from_pandas(pd.DataFrame({'text': X_test, 'label': y_test}))

print("Tokenizing training and test datasets...")
train_dataset = train_dataset.map(lambda e: tokenize_function(e['text']), batched=True)
test_dataset = test_dataset.map(lambda e: tokenize_function(e['text']), batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
print("Tokenization complete.")

# Training arguments
print("Setting up training arguments...")
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)
print("Training arguments set.")

# Define Trainer
print("Initializing Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {'accuracy': accuracy_score(p.label_ids, p.predictions.argmax(-1))},
)
print("Trainer initialized.")

# Train and evaluate the model
print("Training the model...")
trainer.train()
print("Training complete.")

# Evaluate the model
print("Evaluating the model...")
metrics = trainer.evaluate()
print(f"Evaluation complete. Accuracy: {metrics['eval_accuracy']:.4f}")

# Get predictions from the test dataset
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)

# Print classification report
report = classification_report(predictions.label_ids, pred_labels, target_names=[k for k, v in rating_mapping.items()])
print("Evaluation complete. Classification Report:")
print(report)

# Function to predict rating
def predict_movie_rating(screenplay):
    screenplay_cleaned = clean_screenplay_text(screenplay)
    inputs = tokenizer(screenplay_cleaned, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    predicted_class_id = outputs.logits.argmax().item()
    return {v: k for k, v in rating_mapping.items()}[predicted_class_id]

# Example usage
print("Predicting rating for a new screenplay...")
new_screenplay_text, actual_rating = df_aus[["screenplay", "age restrict aus"]].iloc[0]
predicted_rating = predict_movie_rating(new_screenplay_text)
print(f"Predicted Rating: {predicted_rating}")
print(f"Actual Rating: {actual_rating}")


Cleaning screenplay texts...
Sample cleaned screenplay: A   Steve Koren Will Ferrell  Chris Kattan       As we hear What is Love by   night falls and partytime begins.  ., 1103    Coconut Teaser, The Palace, The Roxbury, Tatou, etc.       Of random dancers  gyrating, flirting, making out, drinking.      The  a crowded dance floor  and  the rhythmically swaying backs of...  Our heroes. In their minds, Steve is tall, dark and handsome and  is a little genius. Neither is correct  except for the tall and little part. They simultaneously turn and scope the room. In unison, their heads bop to the . Doug steps out from the bar.  to O.S. female Hey! You want to dance? No? Yes? Alright, don't worry about it. Doug, rejected, steps back as Steve steps out.  to O.S. female Do you want to dance? You do? You don't? Not a problem. They are no strangers to rejection, so neither is fazed. Doug enthusiastically steps towards two attractive girls.  Hey, you wanna...? Two attractive girls turn their back

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...
Converting data to Dataset format...
Tokenizing training and test datasets...


Map:   0%|          | 0/4206 [00:00<?, ? examples/s]

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

...

Trainer initialized.
Training the model...


  0%|          | 0/1578 [00:00<?, ?it/s]

{'loss': 1.8282, 'grad_norm': 4.623781681060791, 'learning_rate': 4.9683143219264897e-05, 'epoch': 0.02}
{'loss': 1.7938, 'grad_norm': 6.057514667510986, 'learning_rate': 4.9366286438529784e-05, 'epoch': 0.04}
{'loss': 1.7966, 'grad_norm': 7.101891994476318, 'learning_rate': 4.904942965779468e-05, 'epoch': 0.06}
{'loss': 1.7892, 'grad_norm': 8.391545295715332, 'learning_rate': 4.873257287705957e-05, 'epoch': 0.08}
{'loss': 1.806, 'grad_norm': 6.778666973114014, 'learning_rate': 4.8415716096324467e-05, 'epoch': 0.1}
{'loss': 1.7595, 'grad_norm': 13.150840759277344, 'learning_rate': 4.8098859315589354e-05, 'epoch': 0.11}
{'loss': 1.7454, 'grad_norm': 7.723327159881592, 'learning_rate': 4.778200253485425e-05, 'epoch': 0.13}
{'loss': 1.7501, 'grad_norm': 9.963008880615234, 'learning_rate': 4.746514575411914e-05, 'epoch': 0.15}
{'loss': 1.7482, 'grad_norm': 5.462363243103027, 'learning_rate': 4.714828897338403e-05, 'epoch': 0.17}
{'loss': 1.7407, 'grad_norm': 4.786800384521484, 'learning_ra

  0%|          | 0/48 [00:00<?, ?it/s]

{'eval_loss': 1.707382321357727, 'eval_accuracy': 0.3324538258575198, 'eval_runtime': 69.1333, 'eval_samples_per_second': 5.482, 'eval_steps_per_second': 0.694, 'epoch': 1.0}
{'loss': 0.603, 'grad_norm': 7.2404069900512695, 'learning_rate': 3.320659062103929e-05, 'epoch': 1.01}
{'loss': 0.4112, 'grad_norm': 8.346576690673828, 'learning_rate': 3.288973384030418e-05, 'epoch': 1.03}
{'loss': 0.3188, 'grad_norm': 1.8949519395828247, 'learning_rate': 3.257287705956908e-05, 'epoch': 1.05}
{'loss': 0.5176, 'grad_norm': 9.921331405639648, 'learning_rate': 3.225602027883397e-05, 'epoch': 1.06}
{'loss': 0.3389, 'grad_norm': 9.63318920135498, 'learning_rate': 3.193916349809886e-05, 'epoch': 1.08}
{'loss': 0.4713, 'grad_norm': 7.865534782409668, 'learning_rate': 3.162230671736375e-05, 'epoch': 1.1}
{'loss': 0.3831, 'grad_norm': 38.90399932861328, 'learning_rate': 3.1305449936628644e-05, 'epoch': 1.12}


In [ ]:
# Custom accuracy function that considers predictions correct if the difference is at most 1
def custom_accuracy(y_true, y_pred):
    correct = sum(abs(y_t - y_p) <= 1 for y_t, y_p in zip(y_true, y_pred))
    return correct / len(y_true)

# Custom precision, recall, and F1-score functions for relaxed criteria
def custom_f1_score(y_true, y_pred, label):
    # True positives: predictions within 1 of the actual value
    tp = sum(abs(y_t - y_p) <= 1 and y_t == label for y_t, y_p in zip(y_true, y_pred))
    
    # False positives: predicted as this label but the actual value is more than 1 away
    fp = sum(abs(y_t - y_p) > 1 and y_p == label for y_t, y_p in zip(y_true, y_pred))
    
    # False negatives: actual value is this label, but the predicted value is more than 1 away
    fn = sum(abs(y_t - y_p) > 1 and y_t == label for y_t, y_p in zip(y_true, y_pred))
    
    # Compute precision, recall, and F1 score
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# Get predictions from the test dataset
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_test = predictions.label_ids

# Evaluating model performance
print("Evaluating model performance...")

# Use custom accuracy instead of default accuracy_score
accuracy = custom_accuracy(y_test, y_pred)
print(f"Custom Accuracy: {accuracy:.4f}")

# Get unique labels
unique_labels = np.unique(y_test)

# Calculate and print custom F1 score for each label
for label in unique_labels:
    precision, recall, f1 = custom_f1_score(y_test, y_pred, label)
    print(f"Label: {label}")
    print(f"  Custom Precision: {precision:.4f}")
    print(f"  Custom Recall: {recall:.4f}")
    print(f"  Custom F1-Score: {f1:.4f}\n")

# Classification report for comparison (using the original metric)
print("\nClassification Report:\n", classification_report(
    y_test, y_pred, target_names=list(rating_mapping.keys()))
)

# Function to predict rating with the updated model
def predict_movie_rating(screenplay):
    screenplay_cleaned = clean_screenplay_text(screenplay)
    inputs = tokenizer(screenplay_cleaned, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    predicted_class_id = outputs.logits.argmax().item()
    return {v: k for k, v in rating_mapping.items()}[predicted_class_id]

# Example usage
print("Predicting rating for a new screenplay...")
new_screenplay_text, actual_rating = df_aus[["screenplay", "age restrict aus"]].iloc[0]
predicted_rating = predict_movie_rating(new_screenplay_text)
print(f"Predicted Rating: {predicted_rating}")
print(f"Actual Rating: {actual_rating}")

In [ ]:
# Save the model
output_dir = './saved_model'
print(f"Saving model to {output_dir}...")

# Save model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("Model and tokenizer saved successfully.")

In [ ]:
# Function to predict rating
def predict_movie_rating(screenplay):
    screenplay_cleaned = clean_screenplay_text(screenplay)
    inputs = tokenizer(screenplay_cleaned, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    predicted_class_id = outputs.logits.argmax().item()
    return {v: k for k, v in rating_mapping.items()}[predicted_class_id]

# Example usage
print("Predicting rating for a new screenplay...")
new_screenplay_text, actual_rating = df_aus[["screenplay", "age restrict aus"]].iloc[0]
predicted_rating = predict_movie_rating(new_screenplay_text)
print(f"Predicted Rating: {predicted_rating}")
print(f"Actual Rating: {actual_rating}")